In [3]:
from bokeh.io import vform
from bokeh.models import CustomJS, ColumnDataSource, Select
from bokeh.plotting import figure, output_file, show, hplot, vplot
from bokeh.models import ColumnDataSource, HBox, VBoxForm,Range1d, HoverTool, BoxZoomTool, \
ResetTool, ResizeTool, PreviewSaveTool, PanTool, BoxSelectTool, LassoSelectTool, Paragraph, CustomJS
from bokeh.models.widgets import CheckboxGroup, Panel, Tabs
from astropy.table import Table
import numpy as np
import pandas as pd
from bokeh.models import Range1d
from bokeh.palettes import RdPu9
import matplotlib.pyplot as plt



TOOLS= [BoxZoomTool(), ResetTool(), ResizeTool(), PanTool(), HoverTool(tooltips=[("sobject_id","@sobject_id")])]

abund=Table.read('/Users/kschles/Documents/GALAH/wg4output/wg4_04292016/sobject_iraf_k2.fits', format='fits')
#abund=Table.read('sobject_iraf_k2.fits', format='fits')
test=Table.to_pandas(abund)
abund_orig=test
test=abund_orig.loc[0:1000]
histcolumns=['sobject_id', 'ra', 'dec', 'teff_cannon', 'logg_cannon', 'feh_cannon', 'alpha_fe_cannon',\
             'teff_sme', 'logg_sme', 'feh_sme', 'alpha_fe_sme', 'teff_guess', 'logg_guess',\
             'feh_guess', 'rv_guess', 'rv_cannon','rv_sme', 'li_abund_sme','c_abund_sme',\
             'o_abund_sme','na_abund_sme','mg_abund_sme','al_abund_sme','si_abund_sme',\
             'k_abund_sme','ca_abund_sme','sc_abund_sme','ti_abund_sme','v_abund_sme',\
             'cr_abund_sme','mn_abund_sme','co_abund_sme','ni_abund_sme','cu_abund_sme',\
             'zn_abund_sme','y_abund_sme','ba_abund_sme','la_abund_sme','nd_abund_sme','eu_abund_sme','rb_abund_sme']

abund=test.loc[:,histcolumns]

output_file("callback.html")

axis_map = {
        "[Fe/H] Cannon": "feh_cannon",
        "[a/Fe] Cannon": "alpha_fe_cannon",
        "Teff Cannon" : "teff_cannon",
        "logg Cannon" : "logg_cannon",
        "[Fe/H] SME" : "feh_sme",
        "[a/Fe] SME": "alpha_fe_sme",
        "Teff SME" : "teff_sme",
        "logg SME" : "logg_sme",
        "RA" : "ra",
        "Dec" : "dec",
        "RV Cannon": "rv_cannon",
        "RV SME" : "rv_sme",
        "RV GUESS" : "rv_guess",
        "[Fe/H] GUESS": "feh_guess",
        "Teff GUESS" : "teff_guess",
        "logg GUESS" : "logg_guess",
        "RV GUESS" : "rv_guess",
        "Li": "li_abund_sme",
        "C" : "c_abund_sme",
        "O" : "o_abund_sme",
        "Na" : "na_abund_sme",
        "Mg" : "mg_abund_sme", 
        "Al" : "al_abund_sme", 
        "Si" : "si_abund_sme", 
        "K"  : "k_abund_sme",
        "Ca" : "ca_abund_sme", 
        "Sc" : "sc_abund_sme",
        "Ti" : "ti_abund_sme", 
        "V"  : "v_abund_sme",
        "Cr" : "cr_abund_sme", 
        "Mn" : "mn_abund_sme",
        "Co" : "co_abund_sme",
        "Ni" : "ni_abund_sme",
        "Cu" : "cu_abund_sme",
        "Zn" : "zn_abund_sme",
        "Y"  : "y_abund_sme",
        "Ba" : "ba_abund_sme",
        "La" : "la_abund_sme",
        "Nd" : "nd_abund_sme",
        "Eu" : "eu_abund_sme",
        "Rb" : "rb_abund_sme"
        }


xname="[Fe/H] Cannon"
yname="[a/Fe] SME"
binval='50'

## Set up initial data: 
x_name = axis_map[xname]
y_name = axis_map[yname]

xbinnum=int(binval)
ybinnum=int(binval)


## Loop through columns in abund to add histograms to dictionary for source call in bokeh interactive.
for i in range(1,len(histcolumns)): 
    colname=histcolumns[i]
    hhist, hedges=np.histogram(abund.loc[np.where(np.isnan(abund[colname])==False)[0],colname], bins=xbinnum)
    lst=[np.nan]*(len(abund)-xbinnum) 
    lst2=[np.nan]*(len(abund)-(xbinnum+1)) 
    name1='hh_'+colname
    name2='hel_'+colname
    name3='her_'+colname
    abund[name1]=(pd.concat([pd.Series(hhist),pd.Series(lst)], ignore_index=True))
    abund[name2]=(pd.concat([pd.Series(hedges[:-1]),pd.Series(lst2)], ignore_index=True))
    abund[name3]=(pd.concat([pd.Series(hedges[1:]),pd.Series(lst2)], ignore_index=True))

abund['x']=abund[x_name]
abund['y']=abund[y_name] 
abund['hhist']=abund['hh_feh_cannon']
abund['hedges_right']=abund['her_feh_cannon'] 
abund['hedges_left']=abund['hel_feh_cannon']
abund['vhist']=abund['hh_alpha_fe_sme']
abund['vedges_right']=abund['her_alpha_fe_sme']
abund['vedges_left']=abund['hel_alpha_fe_sme']
abund['start_x']=np.nan
abund['end_x']=np.nan
abund['start_y']=np.nan
abund['end_y']=np.nan
abund.loc[0,'start_x']=np.min(abund['x'])
abund.loc[0,'end_x']=np.max(abund['x'])
abund.loc[0,'start_y']=np.min(abund['y'])
abund.loc[0,'end_y']=np.max(abund['y'])

        
"""
source = ColumnDataSource(data=dict(x=abund[x_name], y=abund[y_name], hhist=abund['hh_feh_cannon'], \
                                    hedges_right=abund['her_feh_cannon'], hedges_left=abund['hel_feh_cannon'], \
                                    vhist=abund['hh_alpha_fe_sme'], \
                                    vedges_right=abund['her_alpha_fe_sme'], vedges_left=abund['hel_alpha_fe_sme'], \
                                    hh_feh_cannon=abund['hh_feh_cannon'], hel_feh_cannon=abund['hel_feh_cannon'],\
                                    her_feh_cannon=abund['her_feh_cannon'], hh_alpha_fe_sme=abund['hh_alpha_fe_sme'], \
                                    hel_alpha_fe_sme=abund['hel_alpha_fe_sme'], her_alpha_fe_sme=abund['her_alpha_fe_sme'],\
                                    hh_logg_cannon=abund['hh_logg_cannon'], hel_logg_cannon=abund['hel_logg_cannon'], \
                                    her_logg_cannon=abund['her_logg_cannon'], hh_teff_cannon=abund['hh_teff_cannon'], \
                                    hel_teff_cannon=abund['hel_teff_cannon'], her_teff_cannon=abund['her_teff_cannon'],
                                    sobject_id=abund['sobject_id'], \
                                    teff_cannon=abund['teff_cannon'], logg_cannon=abund['logg_cannon'], \
                                    feh_cannon=abund['feh_cannon'], alpha_fe_sme=abund['alpha_fe_sme']))
"""
source = ColumnDataSource(data=dict(abund))


callback = CustomJS(args=dict(source=source), code="""
    
    axis_map = {
            "[Fe/H] Cannon": "feh_cannon",
            "[a/Fe] Cannon": "alpha_fe_cannon",
            "Teff Cannon" : "teff_cannon",
            "logg Cannon" : "logg_cannon",
            "[Fe/H] SME" : "feh_sme",
            "[a/Fe] SME": "alpha_fe_sme",
            "Teff SME" : "teff_sme",
            "logg SME" : "logg_sme",
            "RA" : "ra",
            "Dec" : "dec",
            "RV Cannon": "rv_cannon",
            "RV SME" : "rv_sme",
            "RV GUESS" : "rv_guess",
            "[Fe/H] GUESS": "feh_guess",
            "Teff GUESS" : "teff_guess",
            "logg GUESS" : "logg_guess",
            "RV GUESS" : "rv_guess",
            "Li": "li_abund_sme",
            "C" : "c_abund_sme",
            "O" : "o_abund_sme",
            "Na" : "na_abund_sme",
            "Mg" : "mg_abund_sme", 
            "Al" : "al_abund_sme", 
            "Si" : "si_abund_sme", 
            "K"  : "k_abund_sme",
            "Ca" : "ca_abund_sme", 
            "Sc" : "sc_abund_sme",
            "Ti" : "ti_abund_sme", 
            "V"  : "v_abund_sme",
            "Cr" : "cr_abund_sme", 
            "Mn" : "mn_abund_sme",
            "Co" : "co_abund_sme",
            "Ni" : "ni_abund_sme",
            "Cu" : "cu_abund_sme",
            "Zn" : "zn_abund_sme",
            "Y"  : "y_abund_sme",
            "Ba" : "ba_abund_sme",
            "La" : "la_abund_sme",
            "Nd" : "nd_abund_sme",
            "Eu" : "eu_abund_sme",
            "Rb" : "rb_abund_sme"
        }

    var data = source.get('data');
    var f = cb_obj.get('value')
    x_name=axis_map[f]
    temp = data[x_name]
    x = data['x']
    y = data['y']
    for (i = 0; i < temp.length; i++) {
        x[i] = Math.pow(temp[i], 1)
    }
    start_x = data['start_x']
    end_x = data['end_x']
    start_x[0] = x.min_value
    end_x[0] = x.max_value

    hhist=data['hhist']
    hedges_left=data['hedges_left']
    hedges_right=data['hedges_right']
    name2='hh_'+x_name
    name3='hel_'+x_name
    name4='her_'+x_name
    temp2 = data[name2]
    temp3 = data[name3]
    temp4 = data[name4]
    hhist=data['hhist']
    hedges_left=data['hedges_left']
    hedges_right=data['hedges_right']
    
    for (i=0; i<x.length; i++) { 
        hhist[i]=Math.pow(temp2[i],1)
        hedges_left[i]=Math.pow(temp3[i],1)
        hedges_right[i]=Math.pow(temp4[i],1)
    }
    

    source.trigger('change');
""")

callback2 = CustomJS(args=dict(source=source), code="""
    axis_map = {
            "[Fe/H] Cannon": "feh_cannon",
            "[a/Fe] Cannon": "alpha_fe_cannon",
            "Teff Cannon" : "teff_cannon",
            "logg Cannon" : "logg_cannon",
            "[Fe/H] SME" : "feh_sme",
            "[a/Fe] SME": "alpha_fe_sme",
            "Teff SME" : "teff_sme",
            "logg SME" : "logg_sme",
            "RA" : "ra",
            "Dec" : "dec",
            "RV Cannon": "rv_cannon",
            "RV SME" : "rv_sme",
            "RV GUESS" : "rv_guess",
            "[Fe/H] GUESS": "feh_guess",
            "Teff GUESS" : "teff_guess",
            "logg GUESS" : "logg_guess",
            "RV GUESS" : "rv_guess",
            "Li": "li_abund_sme",
            "C" : "c_abund_sme",
            "O" : "o_abund_sme",
            "Na" : "na_abund_sme",
            "Mg" : "mg_abund_sme", 
            "Al" : "al_abund_sme", 
            "Si" : "si_abund_sme", 
            "K"  : "k_abund_sme",
            "Ca" : "ca_abund_sme", 
            "Sc" : "sc_abund_sme",
            "Ti" : "ti_abund_sme", 
            "V"  : "v_abund_sme",
            "Cr" : "cr_abund_sme", 
            "Mn" : "mn_abund_sme",
            "Co" : "co_abund_sme",
            "Ni" : "ni_abund_sme",
            "Cu" : "cu_abund_sme",
            "Zn" : "zn_abund_sme",
            "Y"  : "y_abund_sme",
            "Ba" : "ba_abund_sme",
            "La" : "la_abund_sme",
            "Nd" : "nd_abund_sme",
            "Eu" : "eu_abund_sme",
            "Rb" : "rb_abund_sme"
        }

    var data = source.get('data');
    var f = cb_obj.get('value')
    y_name=axis_map[f]
    temp = data[y_name]
    x = data['x']
    y = data['y']
    for (i = 0; i < temp.length; i++) {
        y[i] = Math.pow(temp[i], 1)
    }
    start_x = data['start_x']
    end_x = data['end_x']
    start_x[0] = x.min_value
    end_x[0] = x.max_value
 
    vhist=data['vhist']
    vedges_left=data['vedges_left']
    vedges_right=data['vedges_right']
    name2='hh_'+y_name
    name3='hel_'+y_name
    name4='her_'+y_name
    temp2 = data[name2]
    temp3 = data[name3]
    temp4 = data[name4]
    
    for (i=0; i<x.length; i++) { 
        vhist[i]=Math.pow(temp2[i],1)
        vedges_left[i]=Math.pow(temp3[i],1)
        vedges_right[i]=Math.pow(temp4[i],1)
    }
 

    source.trigger('change');
""")

callback3 = CustomJS(args=dict(source=source), code="""
        var data = source.get('data');
        x = data['x']
        y = data['y']
        start_x = data['start_x']
        end_x = data['end_x']
        
        var f = cb_obj.get('value')
        
        if (f == "X axis"){
            start_x[0] = x.max_value;
            end_x[0] = x.min_value;           
        }
        source.trigger('change');
""")

select_x=Select(title="X axis:", value="[Fe/H] Cannon", options=list(np.sort(axis_map.keys())), callback=callback)
select_y=Select(title="Y axis:", value="[a/Fe] SME", options=list(np.sort(axis_map.keys())), callback=callback2)
axes_select=Select(title="Flip:", value='None', options=['None', 'X axis', 'Y axis', 'Both'], callback=callback3)


#scatter_plot.x_range.callback = CustomJS(
#        args=dict(source=source, range=scatter_plot.x_range), code=jscode)


xr = Range1d(start=source.data['start_x'][0],end=source.data['end_x'][0])
yr = Range1d(start=min(abund['y']),end=max(abund['y']))

#scatter_plot = figure(tools=TOOLS, plot_width=600, plot_height=600, title=None, min_border=10, min_border_left=50,y_range=yr, x_range=xr)
scatter_plot = figure(tools=TOOLS, plot_width=600, plot_height=600, title=None, min_border=10, \
                      min_border_left=50)
scatter_plot.scatter('x','y', size=3, source=source, color="#3A5785", alpha=0.6)
#scatter_plot.x_range=('start_x', 'end_x', source=source)

#scatter_plot.x_range.callback = CustomJS(args=dict(source=source), code=jscode % ('start_x', 'end_x'))
tab1 = Panel(child=scatter_plot, title="Scatter Plot")

barf_plot = figure(plot_width=600, plot_height=600, title=None, min_border=10, \
                      min_border_left=50)
barf_plot.scatter('x','y', size=3, source=source, color="#3A5785", alpha=0.6)
tab2 = Panel(child=barf_plot, title="Barf Plot")

## Create the Density plot for the second tab
tabs = Tabs(tabs=[ tab1, tab2 ])

## create the horizontal histogram
LINE_ARGS = dict(color="#3A5785", line_color=None)
ph = figure(toolbar_location=None, plot_width=scatter_plot.plot_width-37, plot_height=200, x_range=scatter_plot.x_range,
                title=None, min_border=10, min_border_left=50)
ph.xgrid.grid_line_color = None
ph.quad(bottom=0, left='hedges_left', right='hedges_right', top='hhist', color="white", line_color="#3A5785", source=source)
ph.min_border_top = 10
ph.min_border_right = 10

## create the vertical histogram
th = 42 ## need to adjust for toolbar height
pv = figure(toolbar_location=None, plot_width=200, plot_height=scatter_plot.plot_height+th-13,
                y_range=scatter_plot.y_range, title=None, min_border=10, min_border_top=th)
pv.ygrid.grid_line_color = None
pv.xaxis.major_label_orientation = -3.14/2
pv.quad(left=0, bottom='vedges_left', top='vedges_right', right='vhist', color="white", line_color="#3A5785", source=source)
pv.min_border_top = 80
pv.min_border_left = 0
 
layout = vform(hplot(select_x, select_y, axes_select), vplot(hplot(tabs, pv), hplot(ph, Paragraph(width=200))), width=800, height=800)
 
show(layout)

In [ ]:
tester=abund_orig[['feh_cannon','alpha_fe_sme']].dropna()
tester.reset_index(drop=True, inplace=True)
counts, xbins, ybins, image=plt.hist2d(tester['feh_cannon'], tester['alpha_fe_sme'], bins=50)
wi=xbins[1]-xbins[0]
he=ybins[1]-xbins[0]
colors = ['#ffffff']+RdPu9[::-1]

xer=[]
yer=[]
counter=[]
color=[]
counts_per_bin=(np.max(counts)-np.min(counts))/8.

for i in range(len(xbins)-1):
    for j in range(len(xbins)-1): 
        xer.append(xbins[i])
        yer.append(ybins[j])
        counter.append(counts[i][j])
        #color.append(colors[min(int(counts[i,j]),9)])
        color.append(colors[int(np.ceil(counts[i,j]/counts_per_bin))])
        
source2=ColumnDataSource(data=dict(xer=xer, yer=yer, color=color, counter=counter, alpha=1.0))

hover2 = HoverTool(
        tooltips=[
            ("(x,y)", "(@xer, @yer)"),
            ("count", "@counter"),
        ]
    )


density_plot = figure(plot_width=900, plot_height=400, tools=[hover2])
density_plot.rect('xer', 'yer', width=wi, height=he, source=source2, color="color")

tab2 = Panel(child=density_plot, title="Density Map")

In [ ]:
tester=abund_orig[['feh_cannon','alpha_fe_sme']].dropna()
tester.reset_index(drop=True, inplace=True)

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib inline

counts, xbins, ybins, image=plt.hist2d(tester['feh_cannon'], tester['alpha_fe_sme'], bins=25)


In [ ]:
print xbins[1]-xbins[0]
print ybins[1]-ybins[0]

In [ ]:
from bokeh.palettes import RdPu9, Greys9
RdPu9

In [ ]:
#colors = RdPu9#+['#ffffff'] #[::-1]
colors = ['#ffffff']+RdPu9[::-1]

In [ ]:
print colors

In [ ]:
print np.shape(xbins), np.shape(ybins), np.shape(counts)

In [ ]:
xer=[]
yer=[]
counter=[]
color=[]
counts_per_bin=(np.max(counts)-np.min(counts))/8.

for i in range(len(xbins)-1):
    for j in range(len(xbins)-1): 
        xer.append(xbins[i])
        yer.append(ybins[j])
        counter.append(counts[i][j])
        #color.append(colors[min(int(counts[i,j]),9)])
        color.append(colors[int(np.ceil(counts[i,j]/counts_per_bin))])
        

In [ ]:
import math
np.max(counts)
min((int(786),8))

counts_per_bin=(np.max(counts)-np.min(counts))/8.
print counts_per_bin
math.ceil(counts[i,j]/counts_per_bin)

In [ ]:
barf=pd.DataFrame()
barf['xer']=xer
barf['yer']=yer
barf['counter']=counter
barf['color']=color
barf['alpha']=1.0


#index0=np.where(barf['color']=='#ffffff')
#index1=np.where(barf['color']=='#fff7f3')
#barf.loc[index1[0],'color']='#ffffff'
#barf.loc[index0[0],'color']='#fff7f3'
#barf.loc[index1[0],'alpha']=0.0

In [ ]:
source=ColumnDataSource(data=dict(xer=xer, yer=yer, color=barf['color'], counter=counter, alpha=barf['alpha']))

In [ ]:
from bokeh.models import HoverTool
from bokeh.plotting import output_notebook

hover = HoverTool(
        tooltips=[
            ("(x,y)", "(@xer, @yer)"),
            ("count", "@counter"),
        ]
    )


p = figure(plot_width=900, plot_height=400, tools=[hover])

#p.rect("xer", "yer", source=source,
#color="color", line_color=None)
p.rect('xer', 'yer', width=0.050, height=0.081, source=source, color="color", fill_alpha=1.0)
p.grid

output_notebook()
show(p)

In [ ]:
print color

In [ ]:
from bokeh.charts import Histogram,Bar, output_file, show, output_notebook

p=figure(plot_width=800, plot_height=800)
p.image(image=[counts], x=0, y=0, dw=10, dh=10, palette='Spectral11')

output_notebook()
show(p)

In [ ]:
from bokeh.embed import components
script, div = components([select_x, select_y, scatter_plot, pv, ph])

div

In [ ]:
f=open('bokeh3.js', 'w')
f.write(script)
f.close()
